In [1]:
from postgis_helpers import PostgreSQL

import sidewalk_gaps
from sidewalk_gaps import CREDENTIALS
from sidewalk_gaps.network_analysis import SidewalkNetwork

schema = "camden"

db = PostgreSQL(
    "sidewalk_gaps",
    verbosity="minimal",
    **CREDENTIALS["localhost"]
)

camden = SidewalkNetwork(db, schema)
camden.setup()
camden.construct_network()
camden.add_pois_to_network()

cemetery
publicbuilding
infrastructure
historic
stadium
wastefacility
carshare


KeyboardInterrupt: 

In [2]:
camden.poi_gdf

,gid,objectid_1,objectid,type,theme,name,state,gdb_geomattr_data,geom,uid,x,y,node_id
0,1419,1420,1501,hospital,Hospital,Saint Marys Hospital,,None,POINT (488742.181 4425064.044),1420,-75.13184,39.97566,11878
1,1120,1121,1204,hospital,Hospital,Pennsylvania Hospital,,None,POINT (486697.173 4421691.030),1121,-75.15572,39.94524,2353
2,3632,3629,3631,hospital,Hospital,Wills Eye Hospital,,None,POINT (486720.757 4421988.441),3577,-75.15545,39.94792,2353
3,10840,10841,13181,hospital,Hospital,Pennsylvania Hospital,,None,POINT (486718.594 4421727.619),10841,-75.15547,39.94557,2353
4,5294,5294,4894,hospital,Hospital,Virtua Camden,New Jersey,None,POINT (490753.637 4419858.148),5295,-75.10821,39.92878,11484
5,408,409,165,hospital,Hospital,Cooper University Hospital,New Jersey,None,POINT (490007.834 4421286.410),409,-75.11696,39.94164,6119


In [3]:
import pandas as pd
poi_node_pairs = pd.merge(camden.poi_gdf,
                                      camden.node_gdf,
                                      left_on='node_id',
                                      right_on='node_id',
                                      how='left',
                                      sort=False,
                                      suffixes=['_from', '_to'])

In [4]:
poi_node_pairs

,gid,objectid_1,objectid,type,theme,name,state,gdb_geomattr_data,geom_from,uid,x_from,y_from,node_id,x_to,y_to,geom_to
0,1419,1420,1501,hospital,Hospital,Saint Marys Hospital,,None,POINT (488742.181 4425064.044),1420,-75.13184,39.97566,11878,-75.125780,39.956102,POINT (489256.560 4422892.525)
1,1120,1121,1204,hospital,Hospital,Pennsylvania Hospital,,None,POINT (486697.173 4421691.030),1121,-75.15572,39.94524,2353,-75.131831,39.941267,POINT (488737.304 4421246.831)
2,3632,3629,3631,hospital,Hospital,Wills Eye Hospital,,None,POINT (486720.757 4421988.441),3577,-75.15545,39.94792,2353,-75.131831,39.941267,POINT (488737.304 4421246.831)
3,10840,10841,13181,hospital,Hospital,Pennsylvania Hospital,,None,POINT (486718.594 4421727.619),10841,-75.15547,39.94557,2353,-75.131831,39.941267,POINT (488737.304 4421246.831)
4,5294,5294,4894,hospital,Hospital,Virtua Camden,New Jersey,None,POINT (490753.637 4419858.148),5295,-75.10821,39.92878,11484,-75.107532,39.928991,POINT (490811.638 4419881.534)
5,408,409,165,hospital,Hospital,Cooper University Hospital,New Jersey,None,POINT (490007.834 4421286.410),409,-75.11696,39.94164,6119,-75.116372,39.942286,POINT (490058.145 4421358.006)


In [5]:
from shapely.geometry import LineString
from geoalchemy2 import Geometry, WKTElement

poi_node_pairs["flow"] = [
                LineString([row["geom_from"], row["geom_to"]]) for idx, row in poi_node_pairs.iterrows()
            ]

In [6]:
poi_node_pairs['geom'] = poi_node_pairs["flow"].apply(
                                        lambda x: WKTElement(x.wkt, srid=26918)
            )

In [7]:
for col in ["flow", "geom_from", "geom_to"]:
                poi_node_pairs.drop(col, inplace=True, axis=1)

In [10]:
poi_node_pairs.geometry.name

'geom'